In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My Drive/15859/matrix-factorization

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/15859/matrix-factorization


In [2]:
import numpy as np
import pandas as pd

In [3]:
filename = 'data/ml-latest-small/ratings.csv'
df = pd.read_csv(filename)
df

In [8]:
A = np.zeros((193609, 610))
for index, row in df.iterrows():
    A[int(row['movieId']-1), int(row['userId']-1)] += 1
with open('data/A.npy', 'wb') as f:
    np.save(f, A)

In [3]:
with open('data/A.npy', 'rb') as f:
    A = np.load(f)

In [4]:
np.linalg.matrix_rank(A)

610

In [ ]:
U, S, VT = np.linalg.svd(A)
print(S)